In [37]:
import pandas as pd

In [38]:
import sys, time
import sqlite3
import json
from io import StringIO
from itertools import islice

In [39]:
# peek the data
chunks = pd.read_json('arxiv-data.json', lines=True, chunksize=5)
peek = next(islice(chunks,0,1))
peek

,abstract,authors,authors_parsed,categories,comments,doi,id,journal-ref,license,report-no,submitter,title,update_date,versions
0,A fully differential calculation in perturba...,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...","[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...",hep-ph,"37 pages, 15 figures; published version",10.1103/PhysRevD.76.013009,704.0001,"Phys.Rev.D76:013009,2007",None,ANL-HEP-PR-07-12,Pavel Nadolsky,Calculation of prompt diphoton production cros...,2008-11-26,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007..."
1,"We describe a new algorithm, the $(k,\ell)$-...",Ileana Streinu and Louis Theran,"[[Streinu, Ileana, ], [Theran, Louis, ]]",math.CO cs.CG,To appear in Graphs and Combinatorics,None,704.0002,None,http://arxiv.org/licenses/nonexclusive-distrib...,None,Louis Theran,Sparsity-certifying Graph Decompositions,2008-12-13,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
2,The evolution of Earth-Moon system is descri...,Hongjun Pan,"[[Pan, Hongjun, ]]",physics.gen-ph,"23 pages, 3 figures",None,704.0003,None,None,None,Hongjun Pan,The evolution of the Earth-Moon system based o...,2008-01-13,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007..."
3,We show that a determinant of Stirling cycle...,David Callan,"[[Callan, David, ]]",math.CO,11 pages,None,704.0004,None,None,None,David Callan,A determinant of Stirling cycle numbers counts...,2007-05-23,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
4,In this paper we show how to compute the $\L...,Wael Abu-Shammala and Alberto Torchinsky,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]",math.CA math.FA,None,None,704.0005,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,2013-10-15,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007..."


In [46]:
# initialize a database
keys = peek.columns.tolist()
keys_converted = ['"'+key+'"' for key in keys]
str_keys = '(' + ','.join(keys_converted) + ')'
# str_vals = '(' + ','.join(['?']*len(keys)) + ')'
print(str_keys)
conn = sqlite3.connect('arxiv.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS arxiv')
cursor.execute('CREATE TABLE arxiv'+str_keys)

("abstract","authors","authors_parsed","categories","comments","doi","id","journal-ref","license","report-no","submitter","title","update_date","versions")


In [47]:
keys

['abstract',
 'authors',
 'authors_parsed',
 'categories',
 'comments',
 'doi',
 'id',
 'journal-ref',
 'license',
 'report-no',
 'submitter',
 'title',
 'update_date',
 'versions']

In [ ]:
# migrate data to database
line = '_'
cnt = 0
num_of_cols = len(keys)
with open('arxiv-data.json', 'r') as f:
    try:
        while line:
            cnt += 1
            line = f.readline()
            line = json.loads(line)
            # insert_sql = 'INSERT INTO arxiv' + str_keys + ' VALUES ' + str_vals
            # cursor.executemany(insert_sql, insert_sql)
            vals = [str(line[key]) for key in keys]
            cursor.execute(r"INSERT INTO arxiv VALUES({})".format(','.join(['?'] * num_of_cols)), vals)
            conn.commit()
    except Exception as e:
        print('Error occurs at: No.{}'.format(cnt))
        print(str_vals)
        print(e)

cursor.close()
conn.close()

In [23]:
conn = sqlite3.connect('arxiv.db')
cursor = conn.cursor()
# cursor.execute('SELECT * FROM arxiv')
# cursor.fetchall()
# test code
cursor.execute('SELECT id, authors FROM arxiv')
result = pd.DataFrame(cursor.fetchall())
result.head()

,0,1
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-..."
1,0704.0002,Ileana Streinu and Louis Theran
2,0704.0003,Hongjun Pan
